In [1]:
%reload_ext autoreload
%autoreload 2

##### Define ChEMBL and Papyrus versions

In [2]:
chembl_version = '31'
papyrus_version = '05.5'
papyrus_flavor = 'nostereo'

##### Define annotation round

In [3]:
annotation_round = 2 # Use only for semi-manual annotation rounds (e.g. round 2)
previous_annotation_round = annotation_round - 1

##### Define data directory

In [4]:
from mutants_in_pcm import data_path
data_path.data_dir = '../data'

##### Import analysis functions

In [18]:
import numpy as np
import pandas as pd
import ast

from mutants_in_pcm.preprocessing import obtain_chembl_data,combine_chembl_papyrus_mutants,merge_chembl_papyrus_mutants
from mutants_in_pcm.annotation import *
from mutants_in_pcm.annotation_check import (read_assay_annotations,read_manual_positive_annotations,
                                             classify_negative_annotations,check_manual_positive_annotations,filter_positive_annotations,filter_negative_annotations)

#### Explore and check automatic annotation from the previous round

In [19]:
# Read assays with previous round automatic annotations
assays_previous_round = read_assay_annotations(chembl_version, previous_annotation_round)
assays_previous_round

,assay_id,description,accession,sequence,mutation,aa_change,mutants,target_id,Protein_Type
0,83907,In vivo inhibitory activity against human Hepa...,Q9Y251,MLLRSKPALPPPLMLLLLGPLGPLSPGALPRPAQAQDVVDLDFFTQ...,NaN,[],[],Q9Y251_WT,WT
1,154606,Inhibitory activity against Palmitoyl-CoA oxid...,P07872,MNPDLRKERASATFNPELITHILDGSPENTRRRREIENLILNDPDF...,NaN,[],[],P07872_WT,WT
2,51352,Inhibition of cytochrome P450 1A2 of isolated ...,P05177,MALSQSVPFSATELLLASAIFCLVFWVLKGLRPRVPKGLKSPPEPW...,NaN,[],[],P05177_WT,WT
3,51895,Inhibition of cytochrome P450 3A4 of isolated ...,P08684,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,NaN,[],[],P08684_WT,WT
4,51528,Inhibition of cytochrome P450 2C9 of isolated ...,P11712,MDSLVVLVLCLSCLLLLSLWRQSSGRGKLPPGPTPLPVIGNILQIG...,NaN,[],[],P11712_WT,WT
...,...,...,...,...,...,...,...,...,...
376228,2136225,KinomeScan assay: inhibition of LATS2,Q9NRM7,MRPKTFPATTYSGNSRQRLQEIREGLKQPSKSSVQGLPAGPNSDTS...,NaN,[],[],Q9NRM7_WT,WT
376229,2136385,KinomeScan assay: inhibition of ROCK1,Q13464,MSTGDSFETRFEKMDNLLRDPKSEVNSDCLLDGLDALVYDLDFPAL...,NaN,[],[],Q13464_WT,WT
376230,2136386,KinomeScan assay: inhibition of ROCK2,O75116,MSRPPPTGKMPGAPETAPGDGAGASRQRKLEALIRDPRSPINVESL...,NaN,[],[],O75116_WT,WT
376231,2136430,KinomeScan assay: inhibition of TGFBR2,P37173,MGRGLLRGLWPLHIVLWTRIASTIPPHVQKSVNNDMIVTDNNGAVK...,NaN,[],[],P37173_WT,WT


In [20]:
# Check newly annotated assays (not previously annotated on ChEMBL)
filter_positive_annotations(assays_previous_round, 'new')

,assay_id,description,accession,sequence,mutation,aa_change,mutants,target_id,Protein_Type
3103,226874,Ratio of IC50 value against hNK1 receptor to t...,P25103,MDNVLPVDSDLSPNISTNTSEPNQFVQPAWQIVLWAAAYTVIVVTS...,NaN,[Q165A],[Q165A],P25103_Q165A,Q165A
5557,153379,Transcriptional activation activity on human I...,Q07869,MVDTESPLCPLSPLEAGDLESPLSEEFLQEMGNIQEISQSIGEDSS...,NaN,[I272F],[I272F],Q07869_I272F,I272F
5558,153380,Transcriptional activation activity on human T...,Q07869,MVDTESPLCPLSPLEAGDLESPLSEEFLQEMGNIQEISQSIGEDSS...,NaN,[T279M],[T279M],Q07869_T279M,T279M
5559,153557,Ratio of transcriptional activation of I272F m...,Q07869,MVDTESPLCPLSPLEAGDLESPLSEEFLQEMGNIQEISQSIGEDSS...,NaN,[I272F],[I272F],Q07869_I272F,I272F
5560,153560,Ratio of transcriptional activation of T279M m...,Q07869,MVDTESPLCPLSPLEAGDLESPLSEEFLQEMGNIQEISQSIGEDSS...,NaN,[T279M],[T279M],Q07869_T279M,T279M
...,...,...,...,...,...,...,...,...,...
371084,2113388,Inhibition of human non-phosphorylated ABL1 F3...,P00519,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...,NaN,[F317L],[F317L],P00519_F317L,F317L
373669,2122982,Inhibition of MLL1 (unknown origin)-mediated H...,Q03164,MAHSCRWRFPARPGTTGGGGGGGRRGLGGAPRQRVPALLLPPGPPV...,NaN,[H3K],[H3K],Q03164_H3K,H3K
374059,2125019,Activation of human PKM2-C424A expressed in Es...,P14618,MSKPHSEAGTAFIQTQQLHAAMADTFLEHMCRLDIDSPPITARNTG...,NaN,[C424A],[C424A],P14618_C424A,C424A
374624,2127601,Binding affinity to human partial length MCL1 ...,Q07820,MFGLKRNAVIGLNLYCGGAGLGAGSGGATRPGGRLLATEKEASARR...,NaN,"[D172N, L2S]",[D172N],Q07820_D172N,D172N


In [21]:
# Check rescued assays (annotated as 'UNDEFINED MUTATION' in ChEMBL)
filter_positive_annotations(assays_previous_round, 'rescued')

,assay_id,description,accession,sequence,mutation,aa_change,mutants,target_id,Protein_Type
225249,1433633,Potentiation of human CFTR F508del/G551D mutan...,P13569,MQRSPLEKASVVSKLFFSWTRPILRKGYRQRLELSDIYQIPSVDSA...,UNDEFINED MUTATION,[G551D],[G551D],P13569_G551D,G551D
227270,1442036,Inhibition of KIT V559/T670I mutant (unknown o...,P10721,MRGARGAWDFLCVLLLLLRVQTGSSQPSVSPGEPSPPSIHPGKSDL...,UNDEFINED MUTATION,[T670I],[T670I],P10721_T670I,T670I
228081,1435798,Inhibition of wild type HIV1 reverse transcrip...,Q72547,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,UNDEFINED MUTATION,"[Y181I, Y181C]","[Y181C, Y181I]",Q72547_Y181C_Y181I,Y181C;Y181I
228390,1440950,Inhibition of EGFR T790M/del746 to 750 mutant ...,P00533,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,UNDEFINED MUTATION,[T790M],[T790M],P00533_T790M,T790M
228616,1441757,Corrector activity at human bronchial epitheli...,P13569,MQRSPLEKASVVSKLFFSWTRPILRKGYRQRLELSDIYQIPSVDSA...,UNDEFINED MUTATION,[G551D],[G551D],P13569_G551D,G551D
...,...,...,...,...,...,...,...,...,...
374540,2127220,Binding affinity to 6His-FLAG-Tev-BRDT (1 to 3...,Q58F21,MSLPSRQTAIIVNPPPPEYINTKKNGRLTNQLQYLQKVVLKDLWKH...,UNDEFINED MUTATION,"[Y309A, Y66A]","[Y66A, Y309A]",Q58F21_Y66A_Y309A,Y66A;Y309A
374554,2127280,Inhibition of 6His-FLAG-Tev-BRDT (1 to 397 res...,Q58F21,MSLPSRQTAIIVNPPPPEYINTKKNGRLTNQLQYLQKVVLKDLWKH...,UNDEFINED MUTATION,"[Y309A, Y66A]","[Y66A, Y309A]",Q58F21_Y66A_Y309A,Y66A;Y309A
374556,2127282,Inhibition of 6His-Thr-BRD3 (1 to 435 residues...,Q15059,MSTATTVAPAGIPATPGPVNPPPPEVSNPSKPGRKTNQLQYMQNVV...,UNDEFINED MUTATION,"[Y348A, Y73A]","[Y73A, Y348A]",Q15059_Y73A_Y348A,Y73A;Y348A
374557,2127283,Inhibition of 6His-Thr-BRD2 (1 to 473 residues...,P25440,MLQNVTPHNKLPGEGNAGLLGLGPEAAAPGKRIRKPSLLYEGFESP...,UNDEFINED MUTATION,"[Y386A, Y113A]","[Y113A, Y386A]",P25440_Y113A_Y386A,Y113A;Y386A


In [47]:
# Check missed annotations by automatic annotation 
filter_negative_annotations(assays_previous_round)

,assay_id,description,accession,sequence,mutation,aa_change,mutants,target_id,Protein_Type
88483,317562,Binding affinity for mutant T877A Androgen rec...,P10275,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,T878A,[T877A],[],P10275_WT,WT
94476,322454,Inhibitory concentration against IL-3 independ...,P10721,MRGARGAWDFLCVLLLLLRVQTGSSQPSVSPGEPSPPSIHPGKSDL...,D816V,[D816V],[],P10721_WT,WT
105052,426971,Ratio of Ki to Km for HCV NS5B RNA polymerase ...,Q8JXU8,RTEEAIYQCCDLDPQARVAIRSLTERLYVGGPLTNSRGENCGYRRX...,S282T,"[S5B, S282T]",[],Q8JXU8_WT,WT
105060,426978,Activity of HCV NS5B RNA polymerase S282T asse...,Q8JXU8,RTEEAIYQCCDLDPQARVAIRSLTERLYVGGPLTNSRGENCGYRRX...,S282T,"[S5B, S282T]",[],Q8JXU8_WT,WT
107598,445385,Agonist activity at GluR1 I687A mutant express...,P42261,MQHIFAFFCTGFLGAVVGANFPNNIQIGGLFPNQQSQEHAAFRFAL...,I705A,[I687A],[],P42261_WT,WT
...,...,...,...,...,...,...,...,...,...
373946,2124228,Stabilization of TTR V3OM mutant (unknown orig...,P02766,MASHRLLLLCLAGLVFVSEAGPTGTGESKCPLMVKVLDAVRGSPAI...,V50M,[V3O],[],P02766_WT,WT
373947,2124230,Stabilization of TTR V3OM mutant (unknown orig...,P02766,MASHRLLLLCLAGLVFVSEAGPTGTGESKCPLMVKVLDAVRGSPAI...,V50M,[V3O],[],P02766_WT,WT
373949,2124234,Binding affinity to TTR V30M mutant (unknown o...,P02766,MASHRLLLLCLAGLVFVSEAGPTGTGESKCPLMVKVLDAVRGSPAI...,V50M,[V30M],[],P02766_WT,WT
375931,2135003,Protac activity at CRBN/AR T878A mutant in hum...,Q96SW2,MAGEGDQQDAAHNMGNHLPLLPAESEEEDEMEVEDQDSKEAKKPNI...,T878A,[T878A],[],Q96SW2_WT,WT


##### Check automatic annotations that were not present in ChEMBL and manually flagged as false positives

The annotation of false positives was done by manually checking the new annotations and "rescued" UNDEFINED
MUTATION assay-accession pairs (651 and 182 in round 1). The assays descriptions were checked against the annotated
mutations. Assay-accession pairs that were considered false positives were extracted in 
anotation_analysis_dir\new_annotations_incorrect.xlsx and given a
"reason" flag that described that reason why the annotated mutation was likely given. This should help categorize the
 false positives and give ideas to improve the annotation pipeline. The different reasons were further grouped into
 more general groups in "group_reason" flag.

In [14]:
false_positives = read_manual_positive_annotations(chembl_version, previous_annotation_round)
false_positives

File with manual annotations for false positives found. Continuing...


,assay_id,accession,mutants,reason,N,group_reason
0,208643,P25103,Sar9M,assay,1,assay
1,801913,A4HSF7,A67I,extract,1,assay
2,801916,A4HSF7,A67I,extract,1,assay
3,801918,A4HSF7,A67I,extract,1,assay
4,1791091,P11678,[H2O],H2O2,1,assay
...,...,...,...,...,...,...
296,1881081,P10721,V559D,missing mutation,1,missing mutation
297,1887622,P10721,V559D,missing mutation,1,missing mutation
298,1888099,P10721,V559D,missing mutation,1,missing mutation
299,1889648,P10721,V559D,missing mutation,1,missing mutation


##### Check ChEMBL annotations that were missed in the automatic annotation and their likely reasons for rejection

The annotation of false negatives was done by classifying all rejected ChEMBL mutations into different groups (i.e.
"rejection_flag") defined from an initial visual inspection and talks to the variant annotation team in ChEMBL.
Further relevance/confidence of the different categories were given by further visual inspection.

In [23]:
# Read negative assays with false negative rejection flags given by automatic classification
false_negatives = classify_negative_annotations(chembl_version, previous_annotation_round)
false_negatives

,assay_id,description_x,accession,sequence_x,mutation,aa_change,mutants,target_id,Protein_Type,doc_id,...,component_id.1,component_type,sequence_y,sequence_md5sum,description_y.1,tax_id.1,organism.1,db_source,db_version,rejection_flag
0,317562,Binding affinity for mutant T877A Androgen rec...,P10275,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,T878A,['T877A'],[],P10275_WT,WT,20674,...,187,PROTEIN,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,0e8126a2b4e057dff4e5e448802e64bf,Androgen receptor,9606,Homo sapiens,SWISS-PROT,2022_02,original_shift_exception
1,322454,Inhibitory concentration against IL-3 independ...,P10721,MRGARGAWDFLCVLLLLLRVQTGSSQPSVSPGEPSPPSIHPGKSDL...,D816V,['D816V'],[],P10721_WT,WT,30197,...,255,PROTEIN,MRGARGAWDFLCVLLLLLRVQTGSSQPSVSPGEPSPPSIHPGKSDL...,f753f2b2d9752a30646c83e8630d1df8,Mast/stem cell growth factor receptor Kit,9606,Homo sapiens,SWISS-PROT,2022_02,original_not_valid
2,426971,Ratio of Ki to Km for HCV NS5B RNA polymerase ...,Q8JXU8,RTEEAIYQCCDLDPQARVAIRSLTERLYVGGPLTNSRGENCGYRRX...,S282T,"['S5B', 'S282T']",[],Q8JXU8_WT,WT,35099,...,3671,PROTEIN,RTEEAIYQCCDLDPQARVAIRSLTERLYVGGPLTNSRGENCGYRRX...,3f5012b8af293164261075aeb623fec5,RNA-directed RNA polymerase,11103,Hepatitis C virus,TREMBL,2022_02,original_not_valid
3,426978,Activity of HCV NS5B RNA polymerase S282T asse...,Q8JXU8,RTEEAIYQCCDLDPQARVAIRSLTERLYVGGPLTNSRGENCGYRRX...,S282T,"['S5B', 'S282T']",[],Q8JXU8_WT,WT,35099,...,3671,PROTEIN,RTEEAIYQCCDLDPQARVAIRSLTERLYVGGPLTNSRGENCGYRRX...,3f5012b8af293164261075aeb623fec5,RNA-directed RNA polymerase,11103,Hepatitis C virus,TREMBL,2022_02,original_not_valid
4,445385,Agonist activity at GluR1 I687A mutant express...,P42261,MQHIFAFFCTGFLGAVVGANFPNNIQIGGLFPNQQSQEHAAFRFAL...,I705A,['I687A'],[],P42261_WT,WT,36637,...,4996,PROTEIN,MQHIFAFFCTGFLGAVVGANFPNNIQIGGLFPNQQSQEHAAFRFAL...,1875787bc6f3513d65588f76d95dede6,Glutamate receptor 1,9606,Homo sapiens,SWISS-PROT,2022_02,original_shift_exception
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,2124228,Stabilization of TTR V3OM mutant (unknown orig...,P02766,MASHRLLLLCLAGLVFVSEAGPTGTGESKCPLMVKVLDAVRGSPAI...,V50M,['V3O'],[],P02766_WT,WT,120955,...,1517,PROTEIN,MASHRLLLLCLAGLVFVSEAGPTGTGESKCPLMVKVLDAVRGSPAI...,6b2e8f183e8bc0d7efc3e9933236616e,Transthyretin,9606,Homo sapiens,SWISS-PROT,2022_02,original_exception_Autocuration
1596,2124230,Stabilization of TTR V3OM mutant (unknown orig...,P02766,MASHRLLLLCLAGLVFVSEAGPTGTGESKCPLMVKVLDAVRGSPAI...,V50M,['V3O'],[],P02766_WT,WT,120955,...,1517,PROTEIN,MASHRLLLLCLAGLVFVSEAGPTGTGESKCPLMVKVLDAVRGSPAI...,6b2e8f183e8bc0d7efc3e9933236616e,Transthyretin,9606,Homo sapiens,SWISS-PROT,2022_02,original_exception_Autocuration
1597,2124234,Binding affinity to TTR V30M mutant (unknown o...,P02766,MASHRLLLLCLAGLVFVSEAGPTGTGESKCPLMVKVLDAVRGSPAI...,V50M,['V30M'],[],P02766_WT,WT,120955,...,1517,PROTEIN,MASHRLLLLCLAGLVFVSEAGPTGTGESKCPLMVKVLDAVRGSPAI...,6b2e8f183e8bc0d7efc3e9933236616e,Transthyretin,9606,Homo sapiens,SWISS-PROT,2022_02,original_shift_exception
1598,2135003,Protac activity at CRBN/AR T878A mutant in hum...,Q96SW2,MAGEGDQQDAAHNMGNHLPLLPAESEEEDEMEVEDQDSKEAKKPNI...,T878A,['T878A'],[],Q96SW2_WT,WT,121349,...,16927,PROTEIN,MAGEGDQQDAAHNMGNHLPLLPAESEEEDEMEVEDQDSKEAKKPNI...,6322efb1bd9f9859167c6800b30ed7ea,Protein cereblon,9606,Homo sapiens,SWISS-PROT,2022_02,original_not_valid


In [24]:
# Check assays in specific rejection flags
false_negatives[false_negatives['rejection_flag'] == 'original_not_valid']

,assay_id,description_x,accession,sequence_x,mutation,aa_change,mutants,target_id,Protein_Type,doc_id,...,component_id.1,component_type,sequence_y,sequence_md5sum,description_y.1,tax_id.1,organism.1,db_source,db_version,rejection_flag
1,322454,Inhibitory concentration against IL-3 independ...,P10721,MRGARGAWDFLCVLLLLLRVQTGSSQPSVSPGEPSPPSIHPGKSDL...,D816V,['D816V'],[],P10721_WT,WT,30197,...,255,PROTEIN,MRGARGAWDFLCVLLLLLRVQTGSSQPSVSPGEPSPPSIHPGKSDL...,f753f2b2d9752a30646c83e8630d1df8,Mast/stem cell growth factor receptor Kit,9606,Homo sapiens,SWISS-PROT,2022_02,original_not_valid
2,426971,Ratio of Ki to Km for HCV NS5B RNA polymerase ...,Q8JXU8,RTEEAIYQCCDLDPQARVAIRSLTERLYVGGPLTNSRGENCGYRRX...,S282T,"['S5B', 'S282T']",[],Q8JXU8_WT,WT,35099,...,3671,PROTEIN,RTEEAIYQCCDLDPQARVAIRSLTERLYVGGPLTNSRGENCGYRRX...,3f5012b8af293164261075aeb623fec5,RNA-directed RNA polymerase,11103,Hepatitis C virus,TREMBL,2022_02,original_not_valid
3,426978,Activity of HCV NS5B RNA polymerase S282T asse...,Q8JXU8,RTEEAIYQCCDLDPQARVAIRSLTERLYVGGPLTNSRGENCGYRRX...,S282T,"['S5B', 'S282T']",[],Q8JXU8_WT,WT,35099,...,3671,PROTEIN,RTEEAIYQCCDLDPQARVAIRSLTERLYVGGPLTNSRGENCGYRRX...,3f5012b8af293164261075aeb623fec5,RNA-directed RNA polymerase,11103,Hepatitis C virus,TREMBL,2022_02,original_not_valid
15,473343,Binding affinity to NR2B receptor S645A/i641A ...,Q13224,MKPRAECCSPKFWLVLAVLAVSGSRARSQKSPPSIGIAVILVGTSD...,"S645A,I641A","['R2B', 'S645A']",['S645A'],Q13224_S645A,S645A,39129,...,59,PROTEIN,MKPRAECCSPKFWLVLAVLAVSGSRARSQKSPPSIGIAVILVGTSD...,5bd7a79d95518f600df9ac85dd9d542c,"Glutamate receptor ionotropic, NMDA 2B",9606,Homo sapiens,SWISS-PROT,2022_02,original_not_valid
19,534387,Inhibition of cross-linking of human Pgp TM3T1...,P08183,MDLEGDRNGGAKKKNFFKLNNKSEKDKKEKKPTVSVFSMFRYSNWL...,"T199R,L339C,F728C","['M3T', 'L339C', 'F728C']","['L339C', 'F728C']",P08183_L339C_F728C,L339C;F728C,42551,...,2619,PROTEIN,MDLEGDRNGGAKKKNFFKLNNKSEKDKKEKKPTVSVFSMFRYSNWL...,e8c72a9371a2775313fe0714108c5e40,ATP-dependent translocase ABCB1,9606,Homo sapiens,SWISS-PROT,2022_02,original_not_valid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586,2111448,Inhibition of p110alpha E542K mutant/p85alpha ...,P27986,MSAEGYQYRALYDYKKEREEDIDLHLGDILTVNKGSLVALGFSDGQ...,E542K,['E542K'],[],P27986_WT,WT,120456,...,846,PROTEIN,MSAEGYQYRALYDYKKEREEDIDLHLGDILTVNKGSLVALGFSDGQ...,5fcafd869a5a794742e5d699000e29f8,Phosphatidylinositol 3-kinase regulatory subun...,9606,Homo sapiens,SWISS-PROT,2022_02,original_not_valid
1588,2117080,Protac activity at CRBN/EGFR L858R/T790M mutan...,Q96SW2,MAGEGDQQDAAHNMGNHLPLLPAESEEEDEMEVEDQDSKEAKKPNI...,"L858R,T790M","['L858R', 'T790M']",[],Q96SW2_WT,WT,120695,...,16927,PROTEIN,MAGEGDQQDAAHNMGNHLPLLPAESEEEDEMEVEDQDSKEAKKPNI...,6322efb1bd9f9859167c6800b30ed7ea,Protein cereblon,9606,Homo sapiens,SWISS-PROT,2022_02,original_not_valid
1589,2117102,Protac activity at CRBN/EGFR L858R/T790M mutan...,Q96SW2,MAGEGDQQDAAHNMGNHLPLLPAESEEEDEMEVEDQDSKEAKKPNI...,"L858R,T790M","['L858R', 'T790M']",[],Q96SW2_WT,WT,120695,...,16927,PROTEIN,MAGEGDQQDAAHNMGNHLPLLPAESEEEDEMEVEDQDSKEAKKPNI...,6322efb1bd9f9859167c6800b30ed7ea,Protein cereblon,9606,Homo sapiens,SWISS-PROT,2022_02,original_not_valid
1598,2135003,Protac activity at CRBN/AR T878A mutant in hum...,Q96SW2,MAGEGDQQDAAHNMGNHLPLLPAESEEEDEMEVEDQDSKEAKKPNI...,T878A,['T878A'],[],Q96SW2_WT,WT,121349,...,16927,PROTEIN,MAGEGDQQDAAHNMGNHLPLLPAESEEEDEMEVEDQDSKEAKKPNI...,6322efb1bd9f9859167c6800b30ed7ea,Protein cereblon,9606,Homo sapiens,SWISS-PROT,2022_02,original_not_valid


In [25]:
# Check how many rejected fall under each rejection category
false_negatives.groupby(['rejection_flag'])['assay_id'].count()

rejection_flag
no_extraction                      175
original_deletion                   77
original_exception_Autocuration    168
original_not_valid                 248
original_shift_exception           323
protein_family                     609
Name: assay_id, dtype: int64

#### Re-annotate assays based on the flags given for false positives and negatives

In [26]:
new_round_assays = manual_reannotation(chembl_version=chembl_version,
                    annotation_round=annotation_round,
                    correct_false_positives=True,
                    correct_false_negatives=True)
new_round_assays

,assay_id,description,accession,sequence,mutation,aa_change,mutants,target_id,Protein_Type
0,83907,In vivo inhibitory activity against human Hepa...,Q9Y251,MLLRSKPALPPPLMLLLLGPLGPLSPGALPRPAQAQDVVDLDFFTQ...,NaN,[],[],Q9Y251_WT,WT
1,154606,Inhibitory activity against Palmitoyl-CoA oxid...,P07872,MNPDLRKERASATFNPELITHILDGSPENTRRRREIENLILNDPDF...,NaN,[],[],P07872_WT,WT
2,51352,Inhibition of cytochrome P450 1A2 of isolated ...,P05177,MALSQSVPFSATELLLASAIFCLVFWVLKGLRPRVPKGLKSPPEPW...,NaN,[],[],P05177_WT,WT
3,51895,Inhibition of cytochrome P450 3A4 of isolated ...,P08684,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,NaN,[],[],P08684_WT,WT
4,51528,Inhibition of cytochrome P450 2C9 of isolated ...,P11712,MDSLVVLVLCLSCLLLLSLWRQSSGRGKLPPGPTPLPVIGNILQIG...,NaN,[],[],P11712_WT,WT
...,...,...,...,...,...,...,...,...,...
376228,2136225,KinomeScan assay: inhibition of LATS2,Q9NRM7,MRPKTFPATTYSGNSRQRLQEIREGLKQPSKSSVQGLPAGPNSDTS...,NaN,[],[],Q9NRM7_WT,WT
376229,2136385,KinomeScan assay: inhibition of ROCK1,Q13464,MSTGDSFETRFEKMDNLLRDPKSEVNSDCLLDGLDALVYDLDFPAL...,NaN,[],[],Q13464_WT,WT
376230,2136386,KinomeScan assay: inhibition of ROCK2,O75116,MSRPPPTGKMPGAPETAPGDGAGASRQRKLEALIRDPRSPINVESL...,NaN,[],[],O75116_WT,WT
376231,2136430,KinomeScan assay: inhibition of TGFBR2,P37173,MGRGLLRGLWPLHIVLWTRIASTIPPHVQKSVNNDMIVTDNNGAVK...,NaN,[],[],P37173_WT,WT


In [27]:
# Check how many assays have mutant annotations (including undefined mutations)
new_round_assays[~new_round_assays['target_id'].str.contains('WT')]

,assay_id,description,accession,sequence,mutation,aa_change,mutants,target_id,Protein_Type
2008,164502,Inhibitory activity against HIV-1 reverse tran...,Q72547,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,M184V,['M184V'],['M184V'],Q72547_M184V,M184V
2009,164500,Inhibitory activity against HIV-1 reverse tran...,Q72547,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,M184I,['M184I'],['M184I'],Q72547_M184I,M184I
2163,164503,Substrate Activity against HIV-1 reverse trans...,Q72547,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,M184V,['M184V'],['M184V'],Q72547_M184V,M184V
2164,164501,Substrate Activity against HIV-1 reverse trans...,Q72547,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,M184I,['M184I'],['M184I'],Q72547_M184I,M184I
3102,205569,Binding affinity against Gln165Ala mutant type...,P25103,MDNVLPVDSDLSPNISTNTSEPNQFVQPAWQIVLWAAAYTVIVVTS...,Q165A,['Q165A'],['Q165A'],P25103_Q165A,Q165A
...,...,...,...,...,...,...,...,...,...
375937,2135067,Protac activity at CRBN/AR T878A mutant in hum...,Q96SW2,MAGEGDQQDAAHNMGNHLPLLPAESEEEDEMEVEDQDSKEAKKPNI...,T878A,['T878A'],[],Q96SW2_MUTANT,WT
375967,2135214,Inhibition of EGFR L858R mutant (unknown origin),P00533,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,L858R,['L858R'],['L858R'],P00533_L858R,L858R
376193,2135936,Inhibition of GDP-loaded His-tagged KRAS G12C ...,P01116,MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...,G12C,['G12C'],['G12C'],P01116_G12C,G12C
376194,2135937,Inhibition of KRAS G12C mutant in human NCI-H3...,P01116,MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...,G12C,['G12C'],['G12C'],P01116_G12C,G12C


In [33]:
# Check how many assays now have new annotations compared to ChEMBL after reannotation
# (undefined mutations are not considered positive annotations)
filter_positive_annotations(new_round_assays, 'new')

,assay_id,description,accession,sequence,mutation,aa_change,mutants,target_id,Protein_Type
3103,226874,Ratio of IC50 value against hNK1 receptor to t...,P25103,MDNVLPVDSDLSPNISTNTSEPNQFVQPAWQIVLWAAAYTVIVVTS...,NaN,['Q165A'],['Q165A'],P25103_Q165A,Q165A
5557,153379,Transcriptional activation activity on human I...,Q07869,MVDTESPLCPLSPLEAGDLESPLSEEFLQEMGNIQEISQSIGEDSS...,NaN,['I272F'],['I272F'],Q07869_I272F,I272F
5558,153380,Transcriptional activation activity on human T...,Q07869,MVDTESPLCPLSPLEAGDLESPLSEEFLQEMGNIQEISQSIGEDSS...,NaN,['T279M'],['T279M'],Q07869_T279M,T279M
5559,153557,Ratio of transcriptional activation of I272F m...,Q07869,MVDTESPLCPLSPLEAGDLESPLSEEFLQEMGNIQEISQSIGEDSS...,NaN,['I272F'],['I272F'],Q07869_I272F,I272F
5560,153560,Ratio of transcriptional activation of T279M m...,Q07869,MVDTESPLCPLSPLEAGDLESPLSEEFLQEMGNIQEISQSIGEDSS...,NaN,['T279M'],['T279M'],Q07869_T279M,T279M
...,...,...,...,...,...,...,...,...,...
367794,2098264,Inhibition of human GCN2 S808G mutant kinase d...,Q9P2K8,MAGGRGAPGRGRDEPPESYPQRQDHELQALEAIYGADFQDLRPDAC...,NaN,['S808G'],['S808G'],Q9P2K8_S808G,S808G
371084,2113388,Inhibition of human non-phosphorylated ABL1 F3...,P00519,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...,NaN,['F317L'],['F317L'],P00519_F317L,F317L
374059,2125019,Activation of human PKM2-C424A expressed in Es...,P14618,MSKPHSEAGTAFIQTQQLHAAMADTFLEHMCRLDIDSPPITARNTG...,NaN,['C424A'],['C424A'],P14618_C424A,C424A
374624,2127601,Binding affinity to human partial length MCL1 ...,Q07820,MFGLKRNAVIGLNLYCGGAGLGAGSGGATRPGGRLLATEKEASARR...,NaN,"['D172N', 'L2S']",['D172N'],Q07820_D172N,D172N


In [34]:
# Check how many assay now have rescued annotations compared to ChEMBL after reannotation
# (undefined mutations are not considered positive annotations)
filter_positive_annotations(new_round_assays, 'rescued')

,assay_id,description,accession,sequence,mutation,aa_change,mutants,target_id,Protein_Type
225249,1433633,Potentiation of human CFTR F508del/G551D mutan...,P13569,MQRSPLEKASVVSKLFFSWTRPILRKGYRQRLELSDIYQIPSVDSA...,UNDEFINED MUTATION,['G551D'],['G551D'],P13569_G551D,G551D
228081,1435798,Inhibition of wild type HIV1 reverse transcrip...,Q72547,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,UNDEFINED MUTATION,"['Y181I', 'Y181C']","['Y181C', 'Y181I']",Q72547_Y181C_Y181I,Y181C;Y181I
228390,1440950,Inhibition of EGFR T790M/del746 to 750 mutant ...,P00533,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,UNDEFINED MUTATION,['T790M'],['T790M'],P00533_T790M,T790M
228616,1441757,Corrector activity at human bronchial epitheli...,P13569,MQRSPLEKASVVSKLFFSWTRPILRKGYRQRLELSDIYQIPSVDSA...,UNDEFINED MUTATION,['G551D'],['G551D'],P13569_G551D,G551D
231419,1472456,Inhibition of FLT3 ITD D835V mutant (unknown o...,P36888,MPALARDGGQLPLLVVFSAMIFGTITNQDLPVIKCVLINHKNNDSS...,UNDEFINED MUTATION,['D835V'],['D835V'],P36888_D835V,D835V
...,...,...,...,...,...,...,...,...,...
372157,2114787,Inhibition of FLT3 ITD/F691L double mutant (un...,P36888,MPALARDGGQLPLLVVFSAMIFGTITNQDLPVIKCVLINHKNNDSS...,UNDEFINED MUTATION,['F691L'],['F691L'],P36888_F691L,F691L
372171,2114825,Inhibition of FLT3 ITD/D835Y double mutant (un...,P36888,MPALARDGGQLPLLVVFSAMIFGTITNQDLPVIKCVLINHKNNDSS...,UNDEFINED MUTATION,['D835Y'],['D835Y'],P36888_D835Y,D835Y
372172,2114826,Inhibition of FLT3 ITD/D835V double mutant (un...,P36888,MPALARDGGQLPLLVVFSAMIFGTITNQDLPVIKCVLINHKNNDSS...,UNDEFINED MUTATION,['D835V'],['D835V'],P36888_D835V,D835V
373444,2121848,Inhibition of EGFR del19/T790M/C797S triple mu...,P00533,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,UNDEFINED MUTATION,"['T790M', 'C797S']","['T790M', 'C797S']",P00533_T790M_C797S,T790M;C797S


In [35]:
# Check how many annotated assays are annotated with an undefined label
new_round_assays[new_round_assays['target_id'].str.contains('MUTANT')]

,assay_id,description,accession,sequence,mutation,aa_change,mutants,target_id,Protein_Type
94476,322454,Inhibitory concentration against IL-3 independ...,P10721,MRGARGAWDFLCVLLLLLRVQTGSSQPSVSPGEPSPPSIHPGKSDL...,D816V,['D816V'],[],P10721_MUTANT,WT
105052,426971,Ratio of Ki to Km for HCV NS5B RNA polymerase ...,Q8JXU8,RTEEAIYQCCDLDPQARVAIRSLTERLYVGGPLTNSRGENCGYRRX...,S282T,"['S5B', 'S282T']",[],Q8JXU8_MUTANT,WT
105060,426978,Activity of HCV NS5B RNA polymerase S282T asse...,Q8JXU8,RTEEAIYQCCDLDPQARVAIRSLTERLYVGGPLTNSRGENCGYRRX...,S282T,"['S5B', 'S282T']",[],Q8JXU8_MUTANT,WT
114613,473331,Binding affinity to NR2B receptor M3c T647A mu...,Q13224,MKPRAECCSPKFWLVLAVLAVSGSRARSQKSPPSIGIAVILVGTSD...,T647T,"['R2B', 'T647A']",['T647A'],Q13224_MUTANT,T647A
114623,473343,Binding affinity to NR2B receptor S645A/i641A ...,Q13224,MKPRAECCSPKFWLVLAVLAVSGSRARSQKSPPSIGIAVILVGTSD...,"S645A,I641A","['R2B', 'S645A']",['S645A'],Q13224_MUTANT,S645A
...,...,...,...,...,...,...,...,...,...
374554,2127280,Inhibition of 6His-FLAG-Tev-BRDT (1 to 397 res...,Q58F21,MSLPSRQTAIIVNPPPPEYINTKKNGRLTNQLQYLQKVVLKDLWKH...,UNDEFINED MUTATION,"['Y309A', 'Y66A']","['Y66A', 'Y309A']",Q58F21_MUTANT,Y66A;Y309A
374556,2127282,Inhibition of 6His-Thr-BRD3 (1 to 435 residues...,Q15059,MSTATTVAPAGIPATPGPVNPPPPEVSNPSKPGRKTNQLQYMQNVV...,UNDEFINED MUTATION,"['Y348A', 'Y73A']","['Y73A', 'Y348A']",Q15059_MUTANT,Y73A;Y348A
374557,2127283,Inhibition of 6His-Thr-BRD2 (1 to 473 residues...,P25440,MLQNVTPHNKLPGEGNAGLLGLGPEAAAPGKRIRKPSLLYEGFESP...,UNDEFINED MUTATION,"['Y386A', 'Y113A']","['Y113A', 'Y386A']",P25440_MUTANT,Y113A;Y386A
375931,2135003,Protac activity at CRBN/AR T878A mutant in hum...,Q96SW2,MAGEGDQQDAAHNMGNHLPLLPAESEEEDEMEVEDQDSKEAKKPNI...,T878A,['T878A'],[],Q96SW2_MUTANT,WT


In [46]:
# Check how many ChEMBL-annotated mutants are still fully rejected 
# (undefined mutations are not considered negative annotations)
filter_negative_annotations(new_round_assays,undefined_mutations=False)

,assay_id,description,accession,sequence,mutation,aa_change,mutants,target_id,Protein_Type
143911,624186,Inhibition of EGFR Leu858Arg and Thr790Met mut...,P04626,MELAALCRWGLLLALLPPGAASTQVCTGTDMKLRLPASPETHLDML...,"L858R,T790M","['L858R', 'T790M']",[],P04626_WT,WT
143913,624186,Inhibition of EGFR Leu858Arg and Thr790Met mut...,Q15303,MKPATGLWVWVSLLVAAGTVQPSDSQSVCAGTENKLSSLSDLEQQY...,"L858R,T790M","['L858R', 'T790M']",[],Q15303_WT,WT
143914,624186,Inhibition of EGFR Leu858Arg and Thr790Met mut...,P21860,MRANDALQVLGLLFSLARGSEVGNSQAVCPGTLNGLSVTGDAENQY...,"L858R,T790M","['L858R', 'T790M']",['L858R'],P21860_L858R,L858R
143915,624185,Inhibition of EGFR Leu858Arg mutant by HTRF assay,P04626,MELAALCRWGLLLALLPPGAASTQVCTGTDMKLRLPASPETHLDML...,L858R,['L858R'],[],P04626_WT,WT
143917,624185,Inhibition of EGFR Leu858Arg mutant by HTRF assay,Q15303,MKPATGLWVWVSLLVAAGTVQPSDSQSVCAGTENKLSSLSDLEQQY...,L858R,['L858R'],[],Q15303_WT,WT
...,...,...,...,...,...,...,...,...,...
364470,2086921,Synergistic potentiator activity at CFTR F508d...,P13569,MQRSPLEKASVVSKLFFSWTRPILRKGYRQRLELSDIYQIPSVDSA...,F508del,[],[],P13569_WT,WT
364471,2086922,Synergistic potentiator activity at CFTR F508d...,P13569,MQRSPLEKASVVSKLFFSWTRPILRKGYRQRLELSDIYQIPSVDSA...,F508del,[],[],P13569_WT,WT
365662,2094601,Potentiation of CFTR F508del mutant (unknown o...,P13569,MQRSPLEKASVVSKLFFSWTRPILRKGYRQRLELSDIYQIPSVDSA...,F508del,[],[],P13569_WT,WT
365666,2094620,Potentiation of CFTR F508del mutant in human H...,P13569,MQRSPLEKASVVSKLFFSWTRPILRKGYRQRLELSDIYQIPSVDSA...,F508del,[],[],P13569_WT,WT


#### Re-annotate bioactivity data for new round based on manual curation

In [48]:
chembl_dataset_new_round = chembl_annotation(chembl_version=chembl_version, annotation_round=annotation_round)
chembl_dataset_new_round

C:\Users\gorostiolam\AppData\Local\Continuum\anaconda3\envs\mutants-in-pcm\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


,target_id,chembl_id,assay_id,activity_id,accession,pchembl_value,activity_comment,canonical_smiles,mutation,year,sequence,pchembl_value_Mean,Activity_class
0,A0A024AXB9_WT,CHEMBL1230673,[1986298],[20667385],A0A024AXB9,[6.37],[nan],CN[C@@H](C)C(=O)N[C@H](C(=O)N1CC[C@H](C)[C@H]1...,NaN,2020.0,AETDEDHAHWLEARMLLDNIYLQDGLIASLYRPEADKVAAIEGEFK...,6.370000,NaN
1,A0A024AXB9_WT,CHEMBL1332616,"[1986298, 1986301]","[20667410, 20667411]",A0A024AXB9,"[5.77, 6.22]","[nan, nan]",O=C1OC2(c3ccccc31)c1cc(I)c(O)c(I)c1Oc1c2cc(I)c...,NaN,2020.0,AETDEDHAHWLEARMLLDNIYLQDGLIASLYRPEADKVAAIEGEFK...,5.995000,NaN
2,A0A024AXB9_WT,CHEMBL151,[1855280],[19053938],A0A024AXB9,[4.28],[nan],O=c1cc(-c2ccc(O)c(O)c2)oc2cc(O)cc(O)c12,NaN,2020.0,AETDEDHAHWLEARMLLDNIYLQDGLIASLYRPEADKVAAIEGEFK...,4.280000,NaN
3,A0A024AXB9_WT,CHEMBL1535,[1986298],[20667415],A0A024AXB9,[4.04],[nan],CCN(CCO)CCCC(C)Nc1ccnc2cc(Cl)ccc12,NaN,2020.0,AETDEDHAHWLEARMLLDNIYLQDGLIASLYRPEADKVAAIEGEFK...,4.040000,NaN
4,A0A024AXB9_WT,CHEMBL164,"[1855280, 1855317, 1855317]","[19053935, 19053936, 19053937]",A0A024AXB9,"[4.66, 5.89, 6.1]","[nan, nan, nan]",O=c1c(O)c(-c2cc(O)c(O)c(O)c2)oc2cc(O)cc(O)c12,NaN,2020.0,AETDEDHAHWLEARMLLDNIYLQDGLIASLYRPEADKVAAIEGEFK...,5.550000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870743,nan_WT,CHEMBL529,"[582849, 582849, 582849]","[2702686, 2702686, 2702686]",NaN,"[6.52, 6.52, 6.52]","[nan, nan, nan]",CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,NaN,2009.0,UGCCUGGCGGCCGUAGCGCGGUGGUCCCACCUGACCCCAUGCCGAA...,6.520000,NaN
1870744,nan_WT,CHEMBL532,"[576183, 576183, 576183, 576182, 576182, 57618...","[2663798, 2663798, 2663798, 2663799, 2663799, ...",NaN,"[7.54, 7.54, 7.54, 7.96, 7.96, 7.96, 5.72, 5.7...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,NaN,2007.0,UGCCUGGCGGCCGUAGCGCGGUGGUCCCACCUGACCCCAUGCCGAA...,6.418889,NaN
1870745,nan_WT,CHEMBL553222,"[582849, 582849, 582849]","[2702683, 2702683, 2702683]",NaN,"[7.0, 7.0, 7.0]","[nan, nan, nan]",CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,NaN,2009.0,UGCCUGGCGGCCGUAGCGCGGUGGUCCCACCUGACCCCAUGCCGAA...,7.000000,NaN
1870746,nan_WT,CHEMBL557931,"[582849, 582849, 582849]","[2702682, 2702682, 2702682]",NaN,"[5.89, 5.89, 5.89]","[nan, nan, nan]",CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,NaN,2009.0,UGCCUGGCGGCCGUAGCGCGGUGGUCCCACCUGACCCCAUGCCGAA...,5.890000,NaN


In [49]:
# Check how many bioactivity datapoints in ChEMBL are tested in mutants
chembl_dataset_new_round[~chembl_dataset_new_round['target_id'].str.contains('WT')]

,target_id,chembl_id,assay_id,activity_id,accession,pchembl_value,activity_comment,canonical_smiles,mutation,year,sequence,pchembl_value_Mean,Activity_class
257,A0A045ISB3_F161S,CHEMBL4637373,[1992590],[20694452],A0A045ISB3,[5.13],[nan],O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Br)cc1,F161S,2020.0,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,5.130,NaN
258,A0A045ISB3_F161S,CHEMBL4637459,[1992590],[20694450],A0A045ISB3,[5.07],[nan],O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2)cc1,F161S,2020.0,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,5.070,NaN
259,A0A045ISB3_F161S,CHEMBL4638011,[1992590],[20694453],A0A045ISB3,[5.04],[nan],O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2F)cc1,F161S,2020.0,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,5.040,NaN
260,A0A045ISB3_F161S,CHEMBL4647367,[1992590],[20694451],A0A045ISB3,[5.27],[nan],O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Cl)cc1,F161S,2020.0,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,5.270,NaN
261,A0A045ISB3_I203A,CHEMBL4637373,[1992587],[20694440],A0A045ISB3,[7.68],[nan],O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Br)cc1,I203A,2020.0,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,7.680,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1868112,Q9YQ12_V82F,CHEMBL3797292,"[1576372, 1576372, 1822317, 1822317]","[16572620, 16572621, 18916325, 18916333]",Q9YQ12,"[7.62, 7.89, 7.62, 7.89]","[nan, nan, nan, nan]",COc1ccc(S(=O)(=O)N(CC(C)C)[C@@H](O)[C@H](Cc2cc...,V82F,2016.0,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,7.755,NaN
1868113,Q9YQ12_V82F,CHEMBL3799941,"[1576372, 1576372, 1822317, 1822317]","[16572618, 16572619, 18916332, 18916379]",Q9YQ12,"[7.72, 8.0, 8.0, 7.72]","[nan, nan, nan, nan]",COc1ccc(S(=O)(=O)N(CC(C)C)[C@@H](O)[C@H](Cc2cc...,V82F,2016.0,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,7.860,NaN
1868114,Q9YQ12_V82F,CHEMBL4452050,[1854379],[19051909],Q9YQ12,[8.77],[nan],COc1ccc(S(=O)(=O)N(CC(C)C)C[C@H](O)[C@@H](Cc2c...,V82F,2019.0,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,8.770,NaN
1870322,T1QYY3_G140S_Q148H,CHEMBL4779601,[2070418],[22876490],T1QYY3,[7.77],[nan],CN(C)C(=O)C(=O)NC12CCC(CC1)Cn1c2nc(C(=O)NCc2cc...,"G140S,Q148H",2020.0,FLDGIDKAQEDHEKYHSNWRAMASDFNMPPIXAKEIVASCDKCQQK...,7.770,NaN


In [50]:
# Check how many of those are tested on undefined mutants
chembl_dataset_new_round[chembl_dataset_new_round['target_id'].str.contains('MUTANT')]

,target_id,chembl_id,assay_id,activity_id,accession,pchembl_value,activity_comment,canonical_smiles,mutation,year,sequence,pchembl_value_Mean,Activity_class
4394,A5Z252_MUTANT,CHEMBL222813,[1804226],[18813644],A5Z252,[8.12],[nan],CC(=O)N[C@@H]1[C@@H](NC(=N)N)C=C(C(=O)O)O[C@H]...,H274Y,2018.0,MNPNQKIITIGSICMVIGIVSLMLQIGNMISIWVSHSIQTGNQRQA...,8.12,NaN
4395,A5Z252_MUTANT,CHEMBL4276773,[1804226],[18813639],A5Z252,[7.44],[nan],CCC(CC)O[C@@H]1C=C(C(=O)O)C[C@H](NCc2ccc(Sc3cc...,H274Y,2018.0,MNPNQKIITIGSICMVIGIVSLMLQIGNMISIWVSHSIQTGNQRQA...,7.44,NaN
4396,A5Z252_MUTANT,CHEMBL4278505,[1804226],[18813633],A5Z252,[6.04],[nan],CCC(CC)O[C@@H]1C=C(C(=O)O)C[C@H](NCc2ccc(Cc3cc...,H274Y,2018.0,MNPNQKIITIGSICMVIGIVSLMLQIGNMISIWVSHSIQTGNQRQA...,6.04,NaN
4397,A5Z252_MUTANT,CHEMBL4281970,[1804226],[18813635],A5Z252,[4.52],[nan],CCC(CC)O[C@@H]1C=C(C(=O)O)C[C@H](NCc2cccc(OC)c...,H274Y,2018.0,MNPNQKIITIGSICMVIGIVSLMLQIGNMISIWVSHSIQTGNQRQA...,4.52,NaN
4398,A5Z252_MUTANT,CHEMBL4282553,[1804226],[18813637],A5Z252,[6.44],[nan],CCC(CC)O[C@@H]1C=C(C(=O)O)C[C@H](NCc2ccc(Oc3cc...,H274Y,2018.0,MNPNQKIITIGSICMVIGIVSLMLQIGNMISIWVSHSIQTGNQRQA...,6.44,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1826325,Q9UM73_MUTANT,CHEMBL3669148,[1640332],[17701115],Q9UM73,[7.89],[nan],COc1ncc(F)cc1-c1c(C(N)=O)sc2cnc(Nc3cc(C4CCOCC4...,L1196M,2015.0,MGAIGLLWLLPLLLSTAAVGSGMGTGQRAGSPAAGPPLQPREPLSY...,7.89,NaN
1826326,Q9UM73_MUTANT,CHEMBL3669149,[1640332],[17701116],Q9UM73,[7.37],[nan],COc1ncccc1-c1c(C(N)=O)sc2cnc(Nc3cccc(N4CCN(C)C...,L1196M,2015.0,MGAIGLLWLLPLLLSTAAVGSGMGTGQRAGSPAAGPPLQPREPLSY...,7.37,NaN
1826327,Q9UM73_MUTANT,CHEMBL3669150,[1640332],[17701117],Q9UM73,[6.37],[nan],COc1ccccc1-c1c(C(N)=O)sc2cnc(Nc3cc4c(cc3OC(C)C...,L1196M,2015.0,MGAIGLLWLLPLLLSTAAVGSGMGTGQRAGSPAAGPPLQPREPLSY...,6.37,NaN
1826328,Q9UM73_MUTANT,CHEMBL3669151,[1640332],[17701118],Q9UM73,[7.54],[nan],COc1ccc(F)cc1-c1c(CO)sc2cnc(Nc3ccc(N4CCN(C)CC4...,L1196M,2015.0,MGAIGLLWLLPLLLSTAAVGSGMGTGQRAGSPAAGPPLQPREPLSY...,7.54,NaN


In [51]:
# Concatenate newly annotated ChEMBL bioactivity data with non-ChEMBL part of Papyrus (only targets with at least
# one annotated variant)
chembl_papyrus_bioactivity_dataset_new_round = combine_chembl_papyrus_mutants(chembl_version, papyrus_version, papyrus_flavor, 1_000_000, annotation_round)
chembl_papyrus_bioactivity_dataset_new_round

C:\Users\gorostiolam\AppData\Local\Continuum\anaconda3\envs\mutants-in-pcm\lib\site-packages\ipykernel_launcher.py:3: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until


,CID,connectivity,target_id,AID,accession,pchembl_value_Mean,SMILES,sequence,source,Activity_class,Year
0,CHEMBL4637373,LUVQLVJTOZVLDB,A0A045ISB3_F161S,[1992590],A0A045ISB3,5.130,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Br)cc1,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,ChEMBL31,NaN,2020.0
1,CHEMBL4637459,ALIJNJWQOZKBPP,A0A045ISB3_F161S,[1992590],A0A045ISB3,5.070,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2)cc1,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,ChEMBL31,NaN,2020.0
2,CHEMBL4638011,IPTNWUSXZLVWFT,A0A045ISB3_F161S,[1992590],A0A045ISB3,5.040,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2F)cc1,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,ChEMBL31,NaN,2020.0
3,CHEMBL4647367,CXXIUJZSLPZZIM,A0A045ISB3_F161S,[1992590],A0A045ISB3,5.270,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Cl)cc1,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,ChEMBL31,NaN,2020.0
4,CHEMBL4637373,LUVQLVJTOZVLDB,A0A045ISB3_I203A,[1992587],A0A045ISB3,7.680,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Br)cc1,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,ChEMBL31,NaN,2020.0
...,...,...,...,...,...,...,...,...,...,...,...
481316,Christmann2016.compound.1413,VMJFTOSOFDEKTM,P08581_N1100Y,Christmann2016.assay,P08581,8.699,Cn1cc(-c2cnc3ccc4c(cc(CS(=O)(=O)NCc5ncccc5)cc4...,MKAPAVLAPGILVLLFTLVQRSNGECKEALAKSEMNVNMKYQLPNF...,Papyrus05.5_Christmann2016,NaN,2011.0
481317,Christmann2016.compound.1413,VMJFTOSOFDEKTM,P08581_Y1230C,Christmann2016.assay,P08581,9.000,Cn1cc(-c2cnc3ccc4c(cc(CS(=O)(=O)NCc5ncccc5)cc4...,MKAPAVLAPGILVLLFTLVQRSNGECKEALAKSEMNVNMKYQLPNF...,Papyrus05.5_Christmann2016,NaN,2011.0
481318,Christmann2016.compound.1413,VMJFTOSOFDEKTM,P08581_Y1230H,Christmann2016.assay,P08581,9.222,Cn1cc(-c2cnc3ccc4c(cc(CS(=O)(=O)NCc5ncccc5)cc4...,MKAPAVLAPGILVLLFTLVQRSNGECKEALAKSEMNVNMKYQLPNF...,Papyrus05.5_Christmann2016,NaN,2011.0
481319,Christmann2016.compound.1640,WCIGMFCFPXZRMQ,Q5S007_A2016T,Christmann2016.assay,Q5S007,7.091,O=C(Nc1cnccc1)c1cc(-c2ccnc(F)c2)ccc1OCc1ccccc1,MASGSCQGCEEDEETLKKLIVRLNNVQEGKQIETLVQILEDLLVFT...,Papyrus05.5_Christmann2016,NaN,2012.0


In [52]:
annotated_data_new_round = merge_chembl_papyrus_mutants(chembl_version, papyrus_version, papyrus_flavor, 1_000_000, annotation_round)
annotated_data_new_round

,target_id,connectivity,pchembl_value_Mean,Activity_class_consensus,source,SMILES,CID,accession,sequence,Year,UniProtID,Organism,HGNC_symbol
0,A0A045ISB3_F161S,ALIJNJWQOZKBPP,5.07,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2)cc1,CHEMBL4637459,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0,A0A045ISB3_MYCTX,Mycobacterium tuberculosis,NaN
1,A0A045ISB3_F161S,CXXIUJZSLPZZIM,5.27,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Cl)cc1,CHEMBL4647367,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0,A0A045ISB3_MYCTX,Mycobacterium tuberculosis,NaN
2,A0A045ISB3_F161S,IPTNWUSXZLVWFT,5.04,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2F)cc1,CHEMBL4638011,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0,A0A045ISB3_MYCTX,Mycobacterium tuberculosis,NaN
3,A0A045ISB3_F161S,LUVQLVJTOZVLDB,5.13,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Br)cc1,CHEMBL4637373,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0,A0A045ISB3_MYCTX,Mycobacterium tuberculosis,NaN
4,A0A045ISB3_I203A,ALIJNJWQOZKBPP,7.37,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2)cc1,CHEMBL4637459,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0,A0A045ISB3_MYCTX,Mycobacterium tuberculosis,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
455834,Q9YQ12_WT,ZYLARFCKPNSSDA,6.17,"Series([], dtype: object)",ChEMBL31,CC(C)[C@H](NC(=O)COc1ccccc1)C(=O)N[C@@H](Cc1cc...,CHEMBL3331342,Q9YQ12,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,2014.0,Q9YQ12_9HIV1,Human immunodeficiency virus 1,NaN
455835,Q9YQ12_WT,ZYLFWZWRSRMGBQ,10.10,"Series([], dtype: object)",ChEMBL31,Nc1cccc(CN2C(=O)N(Cc3ccc4cn[nH]c4c3)C(Cc3ccccc...,CHEMBL2296983,Q9YQ12,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,2013.0,Q9YQ12_9HIV1,Human immunodeficiency virus 1,NaN
455836,Q9YQ12_WT,ZZCUJLFFPCGLAL,5.51,"Series([], dtype: object)",ChEMBL31,CC1(C)CC[C@]2(C(=O)NCCCCCC(=O)NCC(=O)O)CC[C@]3...,CHEMBL4530451,Q9YQ12,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,2019.0,Q9YQ12_9HIV1,Human immunodeficiency virus 1,NaN
455837,Q9YQ12_WT,ZZGMOZUZSAJAML,6.26,"Series([], dtype: object)",ChEMBL31,CC(C)c1nc(CN(C(=O)N[C@H](C(=O)N[C@H](CC[C@H](C...,CHEMBL3115161,Q9YQ12,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,2014.0,Q9YQ12_9HIV1,Human immunodeficiency virus 1,NaN


In [53]:
# Check how many bioactivity datapoints in ChEMBL + Papyrus are tested in mutants
annotated_data_new_round[~annotated_data_new_round['target_id'].str.contains('WT')]

,target_id,connectivity,pchembl_value_Mean,Activity_class_consensus,source,SMILES,CID,accession,sequence,Year,UniProtID,Organism,HGNC_symbol
0,A0A045ISB3_F161S,ALIJNJWQOZKBPP,5.070,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2)cc1,CHEMBL4637459,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0,A0A045ISB3_MYCTX,Mycobacterium tuberculosis,NaN
1,A0A045ISB3_F161S,CXXIUJZSLPZZIM,5.270,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Cl)cc1,CHEMBL4647367,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0,A0A045ISB3_MYCTX,Mycobacterium tuberculosis,NaN
2,A0A045ISB3_F161S,IPTNWUSXZLVWFT,5.040,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2F)cc1,CHEMBL4638011,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0,A0A045ISB3_MYCTX,Mycobacterium tuberculosis,NaN
3,A0A045ISB3_F161S,LUVQLVJTOZVLDB,5.130,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2Br)cc1,CHEMBL4637373,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0,A0A045ISB3_MYCTX,Mycobacterium tuberculosis,NaN
4,A0A045ISB3_I203A,ALIJNJWQOZKBPP,7.370,"Series([], dtype: object)",ChEMBL31,O=C(O)C(=O)Nc1ccc(C#Cc2ccc(C(F)(F)F)cc2)cc1,CHEMBL4637459,A0A045ISB3,MAVRELPGAWNFRDVADTATALRPGRLFRSSELSRLDDAGRATLRR...,2020.0,A0A045ISB3_MYCTX,Mycobacterium tuberculosis,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
454742,Q9YQ12_V82A,SBBYGVNSLWBXFB,7.455,"Series([], dtype: object)",ChEMBL31,COc1ccc(S(=O)(=O)N(CC(C)C)[C@@H](O)[C@H](Cc2cc...,CHEMBL3799941,Q9YQ12,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,2016.0,Q9YQ12_9HIV1,Human immunodeficiency virus 1,NaN
454743,Q9YQ12_V82A,XHKGWTXTLQTTQA,7.400,"Series([], dtype: object)",ChEMBL31,COc1ccc(S(=O)(=O)N(CC(C)C)[C@@H](O)[C@H](Cc2cc...,CHEMBL3797292,Q9YQ12,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,2016.0,Q9YQ12_9HIV1,Human immunodeficiency virus 1,NaN
454744,Q9YQ12_V82F,MEWAZRJLRMEJDV,8.770,"Series([], dtype: object)",ChEMBL31,COc1ccc(S(=O)(=O)N(CC(C)C)C[C@H](O)[C@@H](Cc2c...,CHEMBL4452050,Q9YQ12,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,2019.0,Q9YQ12_9HIV1,Human immunodeficiency virus 1,NaN
454745,Q9YQ12_V82F,SBBYGVNSLWBXFB,7.860,"Series([], dtype: object)",ChEMBL31,COc1ccc(S(=O)(=O)N(CC(C)C)[C@@H](O)[C@H](Cc2cc...,CHEMBL3799941,Q9YQ12,PQITLWQRPFVTIKIEGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,2016.0,Q9YQ12_9HIV1,Human immunodeficiency virus 1,NaN


In [54]:
# Check how many bioactivity datapoints are tested in undefined mutants
annotated_data_new_round[annotated_data_new_round['target_id'].str.contains('MUTANT')]

,target_id,connectivity,pchembl_value_Mean,Activity_class_consensus,source,SMILES,CID,accession,sequence,Year,UniProtID,Organism,HGNC_symbol
1886,A5Z252_MUTANT,ARAIBEBZBOPLMB,8.12,"Series([], dtype: object)",ChEMBL31,CC(=O)N[C@@H]1[C@@H](NC(=N)N)C=C(C(=O)O)O[C@H]...,CHEMBL222813,A5Z252,MNPNQKIITIGSICMVIGIVSLMLQIGNMISIWVSHSIQTGNQRQA...,2018.0,A5Z252_9INFA,Influenza A virus (A/Turkey/651242/2006(H5N1)),NaN
1887,A5Z252_MUTANT,BWDGROPAFQXXFI,6.44,"Series([], dtype: object)",ChEMBL31,CCC(CC)O[C@@H]1C=C(C(=O)O)C[C@H](NCc2ccc(Oc3cc...,CHEMBL4282553,A5Z252,MNPNQKIITIGSICMVIGIVSLMLQIGNMISIWVSHSIQTGNQRQA...,2018.0,A5Z252_9INFA,Influenza A virus (A/Turkey/651242/2006(H5N1)),NaN
1888,A5Z252_MUTANT,DSRHDFMYUHIMKS,4.57,"Series([], dtype: object)",ChEMBL31,CCC(CC)O[C@@H]1C=C(C(=O)O)C[C@H](NCc2ccc(-c3cc...,CHEMBL4285029,A5Z252,MNPNQKIITIGSICMVIGIVSLMLQIGNMISIWVSHSIQTGNQRQA...,2018.0,A5Z252_9INFA,Influenza A virus (A/Turkey/651242/2006(H5N1)),NaN
1889,A5Z252_MUTANT,ITKUZBOVXCDIPW,6.14,"Series([], dtype: object)",ChEMBL31,CCc1ccc(CN[C@H]2CC(C(=O)O)=C[C@@H](OC(CC)CC)[C...,CHEMBL4285767,A5Z252,MNPNQKIITIGSICMVIGIVSLMLQIGNMISIWVSHSIQTGNQRQA...,2018.0,A5Z252_9INFA,Influenza A virus (A/Turkey/651242/2006(H5N1)),NaN
1890,A5Z252_MUTANT,MCRWKUDKDJFWEE,7.44,"Series([], dtype: object)",ChEMBL31,CCC(CC)O[C@@H]1C=C(C(=O)O)C[C@H](NCc2ccc(Sc3cc...,CHEMBL4276773,A5Z252,MNPNQKIITIGSICMVIGIVSLMLQIGNMISIWVSHSIQTGNQRQA...,2018.0,A5Z252_9INFA,Influenza A virus (A/Turkey/651242/2006(H5N1)),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
451580,Q9UM73_MUTANT,OFVDSACHDVLZDS,8.52,"Series([], dtype: object)",ChEMBL31,COc1ncccc1-c1c(C(N)=O)sc2cnc(Nc3cc(C)c(N4CCN(C...,CHEMBL3669139,Q9UM73,MGAIGLLWLLPLLLSTAAVGSGMGTGQRAGSPAAGPPLQPREPLSY...,2015.0,ALK_HUMAN,Homo sapiens (Human),ALK
451581,Q9UM73_MUTANT,QFALLVRHZMVEMI,7.24,"Series([], dtype: object)",ChEMBL31,COc1ccccc1-c1c(C(N)=O)sc2cnc(Nc3ccc(NC4CCOCC4)...,CHEMBL3669147,Q9UM73,MGAIGLLWLLPLLLSTAAVGSGMGTGQRAGSPAAGPPLQPREPLSY...,2015.0,ALK_HUMAN,Homo sapiens (Human),ALK
451582,Q9UM73_MUTANT,SWRGFJBGNAEJOY,7.54,"Series([], dtype: object)",ChEMBL31,COc1ccc(F)cc1-c1c(CO)sc2cnc(Nc3ccc(N4CCN(C)CC4...,CHEMBL3669151,Q9UM73,MGAIGLLWLLPLLLSTAAVGSGMGTGQRAGSPAAGPPLQPREPLSY...,2015.0,ALK_HUMAN,Homo sapiens (Human),ALK
451583,Q9UM73_MUTANT,URHVQJZEFHMQBZ,7.27,"Series([], dtype: object)",ChEMBL31,COc1ccccc1-c1c(C(N)=O)sc2cnc(Nc3ccc(-c4cnn(C)c...,CHEMBL3669143,Q9UM73,MGAIGLLWLLPLLLSTAAVGSGMGTGQRAGSPAAGPPLQPREPLSY...,2015.0,ALK_HUMAN,Homo sapiens (Human),ALK


In [55]:
# Check the distribution of per source of mutant bioactivity datapoints
annotated_data_new_round[~annotated_data_new_round['target_id'].str.contains('WT')].groupby(['source'])['pchembl_value_Mean'].count()

source
ChEMBL31                               22992
Papyrus05.5_Christmann2016               672
Papyrus05.5_Christmann2016;ChEMBL31     1422
Name: pchembl_value_Mean, dtype: int64

#### Quickly check EGFR as case study

##### Check how many deletions are there

In [56]:
# Extract EGFR deletions from false positives and negatives from round 1 
false_negative_deletions = false_negatives[false_negatives['rejection_flag'] == 'original_deletion']
false_positive_deletions = false_positives[false_positives['reason'] == 'missing deletion']
egfr_deletions = pd.concat([false_positive_deletions[false_positive_deletions['accession']=='P00533'][['assay_id',
                                                                                                       'accession']],
                            false_negative_deletions[false_negative_deletions['accession']=='P00533'][['assay_id',
                                                                                                  'accession']]])
egfr_deletions['deletion'] = True

# Attach deletion flags to round 2 assays (with updated target_id)
chembl_assays_with_egfr_deletions = pd.merge(new_round_assays[new_round_assays['accession']=='P00533'], egfr_deletions,
                                             how='left',on=['assay_id','accession']).fillna(False)

# Check how many assays (from ChEMBL) have hidden deletions per EGFR annotated mutation
chembl_assays_with_egfr_deletions.groupby(['target_id','deletion'])['assay_id'].count()

target_id                 deletion
P00533_A750P              False          2
                          True          26
P00533_C797S              False          7
P00533_C797S_L858R        False          2
P00533_G719C              False         30
P00533_G719S              False         30
P00533_L858R              False        176
                          True           1
P00533_L861Q              False         58
P00533_P753S              False          2
                          True          25
P00533_T790M              False        106
                          True          15
P00533_T790M_C797S        False          2
                          True           5
P00533_T790M_C797S_L858R  False         30
P00533_T790M_L858M        False          1
P00533_T790M_L858R        False        293
P00533_T790M_L861R        False          1
P00533_WT                 False       1810
Name: assay_id, dtype: int64

In [57]:
# Attach ChEMBL data to round 2 assays with deletion flags
chembl_data = obtain_chembl_data(chembl_version)
# Not possible to ChEMBL + Papyrus round 2 (annotated_data_new_round) because it does not have assay_id variable anymore
chembl_data_with_egfr_deletions = pd.merge(chembl_data[chembl_data['accession']=='P00533'][['assay_id','accession',
                                                                                            'chembl_id','mutation']],
                                           chembl_assays_with_egfr_deletions[['assay_id','accession','target_id',
                                                                              'deletion']],
                                           how='left',on=['assay_id','accession']).fillna(False)
# Check how many activity datapoints (from ChEMBL) have hidden deletions per EGFR annotated mutation
chembl_data_with_egfr_deletions.groupby(['target_id','deletion'])['chembl_id'].count()

C:\Users\gorostiolam\AppData\Local\Continuum\anaconda3\envs\mutants-in-pcm\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (4,6,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  


target_id                 deletion
P00533_A750P              False          20
                          True           62
P00533_C797S              False          30
P00533_C797S_L858R        False          34
P00533_G719C              False          91
P00533_G719S              False          82
P00533_L858R              False        1704
                          True            1
P00533_L861Q              False         901
P00533_P753S              False          18
                          True           58
P00533_T790M              False        1144
                          True          108
P00533_T790M_C797S        False           5
                          True           20
P00533_T790M_C797S_L858R  False         145
P00533_T790M_L858M        False          30
P00533_T790M_L858R        False        2757
P00533_T790M_L861R        False           1
P00533_WT                 False       14100
Name: chembl_id, dtype: int64